In [ ]:
# 자동 객체 인식으로 바운딩 박스 생성

import torch
import cv2 as cv
from ultralytics import YOLO
import numpy as np
from pathlib import Path
import random
from datetime import datetime

class AutoObjectDetector:
    """자동 객체 탐지로 바운딩 박스 생성"""
    
    def __init__(self, source_path="trashnet", target_path="dataset"):
        self.source_path = Path(source_path)
        self.target_path = Path(target_path)
        
        self.class_mapping = {
            'glass': 0,
            'metal': 1,
            'paper': 2,
            'plastic': 3,
            'trash': 4
        }
        
        # 폴더 생성
        for split in ["train", "val"]:
            (self.target_path / "images" / split).mkdir(parents=True, exist_ok=True)
            (self.target_path / "labels" / split).mkdir(parents=True, exist_ok=True)
    
    def find_object_bbox(self, image):
        """이미지에서 객체의 실제 바운딩 박스 찾기"""
        h, w = image.shape[:2]
        
        # 1. 그레이스케일 변환
        gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
        
        # 2. 가우시안 블러로 노이즈 제거
        blurred = cv.GaussianBlur(gray, (5, 5), 0)
        
        # 3. 이진화 (배경과 객체 분리)
        _, binary = cv.threshold(blurred, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
        
        # 4. 모폴로지 연산으로 노이즈 제거
        kernel = cv.getStructuringElement(cv.MORPH_RECT, (3, 3))
        cleaned = cv.morphologyEx(binary, cv.MORPH_CLOSE, kernel)
        cleaned = cv.morphologyEx(cleaned, cv.MORPH_OPEN, kernel)
        
        # 5. 컨투어 찾기
        contours, _ = cv.findContours(cleaned, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        
        if not contours:
            # 컨투어가 없으면 중앙 80% 반환
            margin = 0.1
            return (int(w * margin), int(h * margin), 
                   int(w * (1 - margin)), int(h * (1 - margin)))
        
        # 6. 가장 큰 컨투어 찾기 (주요 객체)
        largest_contour = max(contours, key=cv.contourArea)
        
        # 7. 바운딩 박스 계산
        x, y, w_box, h_box = cv.boundingRect(largest_contour)
        
        # 8. 바운딩 박스 확장 (10% 마진 추가)
        margin = 0.1
        expand_x = int(w_box * margin)
        expand_y = int(h_box * margin)
        
        x1 = max(0, x - expand_x)
        y1 = max(0, y - expand_y)
        x2 = min(w, x + w_box + expand_x)
        y2 = min(h, y + h_box + expand_y)
        
        # 9. 너무 작거나 큰 박스 필터링
        box_area = (x2 - x1) * (y2 - y1)
        image_area = w * h
        area_ratio = box_area / image_area
        
        if area_ratio < 0.1 or area_ratio > 0.9:
            # 비정상적인 크기면 중앙 70% 반환
            margin = 0.15
            return (int(w * margin), int(h * margin), 
                   int(w * (1 - margin)), int(h * (1 - margin)))
        
        return (x1, y1, x2, y2)
    
    def process_single_image(self, img_path, class_id, split):
        """이미지 처리 - 자동 객체 탐지"""
        try:
            image = cv.imread(str(img_path))
            if image is None:
                return False
            
            h, w = image.shape[:2]
            
            # 새 파일명
            timestamp = datetime.now().strftime('%H%M%S%f')[:8]
            new_filename = f"{class_id}_{img_path.stem}_{timestamp}.jpg"
            
            # 이미지 저장
            target_img_path = self.target_path / "images" / split / new_filename
            cv.imwrite(str(target_img_path), image)
            
            # 🎯 자동 객체 바운딩 박스 탐지
            x1, y1, x2, y2 = self.find_object_bbox(image)
            
            # YOLO 형식으로 변환 (정규화)
            center_x = (x1 + x2) / (2 * w)
            center_y = (y1 + y2) / (2 * h)
            width = (x2 - x1) / w
            height = (y2 - y1) / h
            
            # 라벨 파일 저장
            label_filename = f"{new_filename.rsplit('.', 1)[0]}.txt"
            label_path = self.target_path / "labels" / split / label_filename
            
            with open(label_path, 'w') as f:
                f.write(f"{class_id} {center_x:.6f} {center_y:.6f} {width:.6f} {height:.6f}\n")
            
            return True
            
        except Exception as e:
            print(f"❌ 처리 실패 {img_path}: {e}")
            return False
    
    def convert_with_preview(self, train_ratio=0.8, show_preview=True):
        """데이터 변환 + 미리보기"""
        print("🔄 자동 객체 탐지로 바운딩 박스 생성 시작...")
        
        total_converted = 0
        
        for folder_name, class_id in self.class_mapping.items():
            folder_path = self.source_path / folder_name
            
            if not folder_path.exists():
                continue
            
            # 이미지 찾기
            images = []
            for ext in ['.jpg', '.jpeg', '.png', '.bmp']:
                images.extend(list(folder_path.glob(f"*{ext}")))
                images.extend(list(folder_path.glob(f"*{ext.upper()}")))
            
            if not images:
                continue
            
            print(f"\n📁 {folder_name}: {len(images)}장 처리 중...")
            
            # 미리보기 (첫 5장)
            if show_preview:
                print("🔍 미리보기 (처음 5장):")
                for i, img_path in enumerate(images[:5]):
                    image = cv.imread(str(img_path))
                    if image is not None:
                        x1, y1, x2, y2 = self.find_object_bbox(image)
                        
                        # 박스 그리기
                        preview = image.copy()
                        cv.rectangle(preview, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        cv.putText(preview, f"{folder_name}", (x1, y1-10),
                                  cv.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                        
                        # 미리보기 저장
                        preview_path = f"preview_{folder_name}_{i+1}.jpg"
                        cv.imwrite(preview_path, preview)
                        print(f"  📸 미리보기 저장: {preview_path}")
            
            # 실제 변환
            random.shuffle(images)
            split_idx = int(len(images) * train_ratio)
            
            train_images = images[:split_idx]
            val_images = images[split_idx:]
            
            converted = 0
            
            # 훈련 데이터 처리
            for img_path in train_images:
                if self.process_single_image(img_path, class_id, "train"):
                    converted += 1
            
            # 검증 데이터 처리
            for img_path in val_images:
                if self.process_single_image(img_path, class_id, "val"):
                    converted += 1
            
            total_converted += converted
            print(f"✅ {folder_name}: {converted}장 변환 완료")
        
        print(f"\n🎉 총 {total_converted}장 자동 변환 완료!")
        print("📸 preview_*.jpg 파일들을 확인해서 바운딩 박스가 정확한지 확인하세요.")
        
        return total_converted


class QuickValidator:
    """빠른 검증 도구"""
    
    def __init__(self, dataset_path="dataset"):
        self.dataset_path = Path(dataset_path)
    
    def validate_random_samples(self, num_samples=10):
        """랜덤 샘플 검증"""
        train_images = list((self.dataset_path / "images" / "train").glob("*.jpg"))
        
        if len(train_images) < num_samples:
            num_samples = len(train_images)
        
        samples = random.sample(train_images, num_samples)
        
        print(f"🔍 랜덤 {num_samples}장 검증 중...")
        
        class_names = ['glass', 'metal', 'paper', 'plastic', 'trash']
        
        for i, img_path in enumerate(samples):
            # 이미지 로드
            image = cv.imread(str(img_path))
            if image is None:
                continue
            
            # 라벨 로드
            label_path = self.dataset_path / "labels" / "train" / f"{img_path.stem}.txt"
            
            if not label_path.exists():
                continue
            
            # 라벨 읽기
            with open(label_path, 'r') as f:
                line = f.readline().strip()
                if line:
                    parts = line.split()
                    class_id = int(parts[0])
                    center_x = float(parts[1])
                    center_y = float(parts[2])
                    width = float(parts[3])
                    height = float(parts[4])
            
            # 바운딩 박스 그리기
            h, w = image.shape[:2]
            x1 = int((center_x - width/2) * w)
            y1 = int((center_y - height/2) * h)
            x2 = int((center_x + width/2) * w)
            y2 = int((center_y + height/2) * h)
            
            cv.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv.putText(image, f"{class_names[class_id]}", (x1, y1-10),
                      cv.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            # 검증 이미지 저장
            validation_path = f"validation_sample_{i+1}_{class_names[class_id]}.jpg"
            cv.imwrite(validation_path, image)
        
        print(f"✅ validation_sample_*.jpg 파일들을 확인하세요!")


def create_yaml():
    """YAML 파일 생성"""
    yaml_content = """path: ./dataset
train: images/train
val: images/val
nc: 5
names: ['glass', 'metal', 'paper', 'plastic', 'trash']
"""
    
    with open("dataset.yaml", "w") as f:
        f.write(yaml_content)
    print("✅ dataset.yaml 생성 완료")


def main():
    print("🎯 자동 객체 탐지 바운딩 박스 생성")
    print("=" * 50)
    
    while True:
        print("\n선택하세요:")
        print("1️⃣ 자동 객체 탐지 + 변환 (미리보기 포함)")
        print("2️⃣ 자동 객체 탐지 + 변환 (미리보기 없음)")  
        print("3️⃣ 결과 검증 (랜덤 샘플 확인)")
        print("4️⃣ YAML 파일 생성")
        print("0️⃣ 종료")
        
        choice = input("선택: ").strip()
        
        if choice == "1":
            source_folder = input("원본 폴더 경로 (기본값: trashnet): ").strip()
            if not source_folder:
                source_folder = "trashnet"
            
            detector = AutoObjectDetector(source_folder)
            detector.convert_with_preview(show_preview=True)
        
        elif choice == "2":
            source_folder = input("원본 폴더 경로 (기본값: trashnet): ").strip()
            if not source_folder:
                source_folder = "trashnet"
                
            detector = AutoObjectDetector(source_folder)
            detector.convert_with_preview(show_preview=False)
        
        elif choice == "3":
            validator = QuickValidator()
            validator.validate_random_samples()
        
        elif choice == "4":
            create_yaml()
        
        elif choice == "0":
            break
        
        else:
            print("❌ 잘못된 선택")


if __name__ == "__main__":
    main()